# MetaSpore Getting Started

MetaSpore is a machine learning platform, which provides a one-stop solution for data preprocessing, model training and online prediction.

In this article, we introduce the basic API of MetaSpore briefly.

## Prepare Data

We use the publicly available dataset [Terabyte Click Logs](https://labs.criteo.com/2013/12/download-terabyte-click-logs-2/) published by CriteoLabs as our demo dataset.

We sample the dataset with sampling rate 0.001 so that the running of the demo can finish quickly. More information about the demo dataset can be found in [MetaSpore Demo Dataset](https://ks3-cn-beijing.ksyuncs.com/dmetasoul-bucket/demo/criteo/index.html).

Execute the following cell to download the demo dataset into the working directory. Those data files take up about 2.1 GiB disk space and the downloading process may take sveral minutes. If the downloading fails, please refer to [MetaSpore Demo Dataset](https://ks3-cn-beijing.ksyuncs.com/dmetasoul-bucket/demo/criteo/index.html) and download the dataset manually.

In [1]:
# import metaspore
# metaspore.demo.download_dataset()

You can check the downloaded dataset by executing the following cell.

In [2]:
!ls -l ${PWD}/data/

total 8
drwxrwxr-x 3 ec2-user ec2-user 4096 Jul  8 09:25 test
drwxrwxr-x 2 ec2-user ec2-user 4096 Jul  6 03:30 train


(Optional) To upload the dataset to your own s3 bucket:

1. Fill ``{YOUR_S3_BUCKET}`` and ``{YOUR_S3_PATH}`` with your preferred values in the following cell.
2. Uncomment the cell by removing the leading ``#`` character.
3. Execute the cell.

In [3]:
YOUR_S3_BUCKET='s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/'
YOUR_S3_PATH='datasets/CriteoLabs'

In [ ]:
# !aws s3 cp --recursive ${PWD}/data/ s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/data/

Alternatively, you can open a terminal by selecting the ``File`` -> ``New`` -> ``Terminal`` menu item and executing Bash commands in it.

You can check the uploaded dataset in your s3 bucket by uncommenting and executing the following cell.

In [13]:
!aws s3 ls s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/data/

                           PRE test/
                           PRE train/


The ``schema`` directory contains configuration files and must also be uploaded to s3 so that the model can be trained in cluster environment. 

In [14]:
#!aws s3 cp --recursive ${PWD}/schema/ s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/schema/

In the rest of the article, we assume the demo dataset and schemas has been uploaded to `ROOT_DIR`.

In [15]:
ROOT_DIR = 's3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo'
# ROOT_DIR = '.'

## Define the Model

We can define our neural network model by subclassing ``torch.nn.Module`` as usual PyTorch models. The following ``DemoModule`` class provides an example.

Compared to usual PyTorch models, the notable difference is the ``_sparse`` layer created by instantiating ``ms.EmbeddingSumConcat`` which takes an embedding size and paths of two text files. ``ms.EmbeddingSumConcat`` makes it possible to define large-scale sparse models in PyTorch, which is a distinguishing feature of MetaSpore.

The ``_schema_dir`` field is an s3 directory which makes it possible to use the ``DemoModule`` class in cluster environment.

In [16]:
import torch
import metaspore as ms

class DemoModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self._embedding_size = 16
        self._schema_dir = ROOT_DIR + '/schema/'
        self._column_name_path = self._schema_dir + 'column_name_demo.txt'
        self._combine_schema_path = self._schema_dir + 'combine_schema_demo.txt'
        self._sparse = ms.EmbeddingSumConcat(self._embedding_size, self._column_name_path, self._combine_schema_path)
        self._sparse.updater = ms.FTRLTensorUpdater()
        self._sparse.initializer = ms.NormalTensorInitializer(var=0.01)
        self._dense = torch.nn.Sequential(
            ms.nn.Normalization(self._sparse.feature_count * self._embedding_size),
            torch.nn.Linear(self._sparse.feature_count * self._embedding_size, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self._sparse(x)
        x = self._dense(x)
        return torch.sigmoid(x)

Instantiating the ``DemoModule`` class to define our PyTorch model.

In [17]:
module = DemoModule()

[WARN] 2024-07-08 09:42:16.048 STSAssumeRoleWithWebIdentityCredentialsProvider [140032615020352] Token file must be specified to use STS AssumeRole web identity creds provider.
[2024-07-08 09:42:16.053] [info] [s3_sdk_filesys.cpp:357] Try to open S3 stream: s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/schema/combine_schema_demo.txt, read_only true
loaded combine schema from combine schema file 's3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/schema/combine_schema_demo.txt'
[2024-07-08 09:42:16.076] [info] [s3_sdk_filesys.cpp:380] Opened read-only stream for object: s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/schema/combine_schema_demo.txt with total length 930
[2024-07-08 09:42:16.080] [info] [s3_sdk_filesys.cpp:419] Read S3 object s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/schema/combine_schema_demo.txt with size 930 at position 0 larger than total size: 930, change size to 930
[2024-07-08 09:42:16.096] [info] [s3

## Train the Model

To train our model, first we need to create a ``ms.PyTorchEstimator`` passing in several arguments including our PyTorch model ``module`` and the number of workers and servers.

``model_out_path`` specifies where to store the trained model.

``input_label_column_index`` specifies the column index of the label column in the dataset, which is ``0`` for the demo dataset.

In [18]:
model_out_path = ROOT_DIR + '/output/dev/model_out/'
estimator = ms.PyTorchEstimator(module=module,
                                worker_count=4,
                                server_count=4,
                                model_out_path=model_out_path,
                                experiment_name='0.1',
                                input_label_column_index=0)

Next, we create a Spark session by calling ``ms.spark.get_session()`` and load the training dataset by call ``ms.input.read_s3_csv()``.

``delimiter`` specifies the column delimiter of the dataset, which is the TAB character ``'\t'`` for the demo dataset.

We also need to pass column names because the csv files do not contain headers.

In [21]:
column_names = []
with open(f'./schema/column_name_demo.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['label', 'integer_feature_1', 'integer_feature_2', 'integer_feature_3', 'integer_feature_4', 'integer_feature_5', 'integer_feature_6', 'integer_feature_7', 'integer_feature_8', 'integer_feature_9', 'integer_feature_10', 'integer_feature_11', 'integer_feature_12', 'integer_feature_13', 'categorical_feature_1', 'categorical_feature_2', 'categorical_feature_3', 'categorical_feature_4', 'categorical_feature_5', 'categorical_feature_6', 'categorical_feature_7', 'categorical_feature_8', 'categorical_feature_9', 'categorical_feature_10', 'categorical_feature_11', 'categorical_feature_12', 'categorical_feature_13', 'categorical_feature_14', 'categorical_feature_15', 'categorical_feature_16', 'categorical_feature_17', 'categorical_feature_18', 'categorical_feature_19', 'categorical_feature_20', 'categorical_feature_21', 'categorical_feature_22', 'categorical_feature_23', 'categorical_feature_24', 'categorical_feature_25', 'categorical_feature_26']


In [30]:
train_dataset_path = ROOT_DIR + '/data/train/' # '/data/train/day_0_0.001_train.csv'

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=estimator.worker_count,
                                     server_count=estimator.server_count,
                                     log_level='INFO',
                                     spark_confs={'spark.eventLog.enabled':'true'})
train_dataset = ms.input.read_s3_csv(spark_session, train_dataset_path, delimiter='\t', column_names=column_names)

24/07/08 14:41:57 INFO SparkContext: Running Spark version 3.1.2
24/07/08 14:41:57 INFO ResourceUtils: ==============================================================
24/07/08 14:41:57 INFO ResourceUtils: No custom resources configured for spark.driver.
24/07/08 14:41:57 INFO ResourceUtils: ==============================================================
24/07/08 14:41:57 INFO SparkContext: Submitted application: MetaSpore-Notebook
24/07/08 14:41:57 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 5120, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/07/08 14:41:57 INFO ResourceProfile: Limiting resource is cpus at 1 tasks per executor
24/07/08 14:41:57 INFO ResourceProfileManager: Added ResourceProfile id: 0
24/07/08 14:41:57 INFO SecurityManager: Changing view acls to: ec2-user
24/07/08 14

ignore shuffle


24/07/08 14:41:58 INFO InMemoryFileIndex: It took 47 ms to list leaf files for 1 paths.


In [31]:
train_dataset.count()

24/07/08 14:42:29 INFO FileSourceStrategy: Pushed Filters: 
24/07/08 14:42:29 INFO FileSourceStrategy: Post-Scan Filters: 
24/07/08 14:42:29 INFO FileSourceStrategy: Output Data Schema: struct<>
24/07/08 14:42:29 INFO CodeGenerator: Code generated in 5.849617 ms
24/07/08 14:42:29 INFO CodeGenerator: Code generated in 4.183812 ms
24/07/08 14:42:29 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 195.9 KiB, free 2.8 GiB)
24/07/08 14:42:29 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 26.8 KiB, free 2.8 GiB)
24/07/08 14:42:29 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ip-172-16-14-249.us-west-2.compute.internal:45547 (size: 26.8 KiB, free: 2.8 GiB)
24/07/08 14:42:29 INFO SparkContext: Created broadcast 0 from count at NativeMethodAccessorImpl.java:0
24/07/08 14:42:29 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/

4375765

In [33]:
train_dataset = train_dataset.limit(500000)

Finally, we call the ``fit()`` method of ``ms.PyTorchEstimator`` to train our model. This will take several minutes and you can see the progress by looking at the output of the cell. The trained model is stored in ``model_out_path`` and the ``model`` variable.

In [34]:
model = estimator.fit(train_dataset)

[2024-07-08 14:48:31.262] [info] PS job with coordinator address 172.16.14.249:51947 started.
[2024-07-08 14:48:31.262] [info] PSRunner::RunPS: pid: 14693, tid: 19386, thread: 0x7f5b90bd9700
[2024-07-08 14:48:31.262] [info] PSRunner::RunPSCoordinator: pid: 14693, tid: 19386, thread: 0x7f5b90bd9700
[2024-07-08 14:48:31.263] [info] ActorProcess::Receiving: Coordinator pid: 14693, tid: 19389, thread: 0x7f5b85fd4700


24/07/08 14:48:31 INFO SparkContext: Starting job: collect at PythonRDD.scala:180
24/07/08 14:48:31 INFO SparkContext: Starting job: collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:308
24/07/08 14:48:31 INFO DAGScheduler: Got job 2 (collect at PythonRDD.scala:180) with 1 output partitions
24/07/08 14:48:31 INFO DAGScheduler: Final stage: ResultStage 3 (collect at PythonRDD.scala:180)
24/07/08 14:48:31 INFO DAGScheduler: Parents of final stage: List()
24/07/08 14:48:31 INFO DAGScheduler: Missing parents: List()
24/07/08 14:48:31 INFO DAGScheduler: Submitting ResultStage 3 (PythonRDD[13] at RDD at PythonRDD.scala:53), which has no missing parents
24/07/08 14:48:31 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 6.8 KiB, free 2.8 GiB)
24/07/08 14:48:31 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 4.2 KiB, free 2.8 GiB)
24/07/08 14:48:31 INFO BlockManagerInfo: Added broa

[2024-07-08 14:48:32.396] [info] C[0]:9: The coordinator has connected to 1 servers and 1 workers.
PS Coordinator node C[0]:9 is ready.


24/07/08 14:48:32 INFO PythonRunner: Times: total = 41, boot = 9, init = 32, finish = 0
24/07/08 14:48:32 INFO Executor: Finished task 0.0 in stage 6.0 (TID 16). 1268 bytes result sent to driver
24/07/08 14:48:32 INFO TaskSetManager: Finished task 0.0 in stage 6.0 (TID 16) in 46 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (1/1)
24/07/08 14:48:32 INFO TaskSchedulerImpl: Removed TaskSet 6.0, whose tasks have all completed, from pool 
24/07/08 14:48:32 INFO DAGScheduler: ResultStage 6 (collect at PythonRDD.scala:180) finished in 0.050 s
24/07/08 14:48:32 INFO DAGScheduler: Job 5 is finished. Cancelling potential speculative or zombie tasks for this job
24/07/08 14:48:32 INFO TaskSchedulerImpl: Killing all running tasks in stage 6: Stage finished
24/07/08 14:48:32 INFO DAGScheduler: Job 5 finished: collect at PythonRDD.scala:180, took 0.051320 s
24/07/08 14:48:32 INFO SparkContext: Starting job: collect at PythonRDD.scala:180
24/07/08 14:48:32 INFO DAGScheduler: Got

2024-07-08 14:49:12.870 -- auc: 0.49693195179306293, Δauc: 0.49693195179306293, pcoc: 2.3028489149042537, Δpcoc: 2.3028489149042537, loss: 0.0016268559694290162, Δloss: 0.0016268559694290162, #instance: 1000
2024-07-08 14:49:13.350 -- auc: 0.44041719317643346, Δauc: 0.37290812841530063, pcoc: 2.792590057047514, Δpcoc: 3.363954722881317, loss: 0.0016051685214042662, Δloss: 0.0015834810733795165, #instance: 2000
2024-07-08 14:49:13.828 -- auc: 0.4484202132961228, Δauc: 0.47188915534239273, pcoc: 2.3739081383505956, Δpcoc: 1.5675577764157895, loss: 0.0015410221020380657, Δloss: 0.001412729263305664, #instance: 3000
2024-07-08 14:49:14.297 -- auc: 0.46016709017717583, Δauc: 0.5119400197443902, pcoc: 1.9826464584823382, Δpcoc: 1.1900907479799712, loss: 0.0016158361732959746, Δloss: 0.0018402783870697021, #instance: 4000
2024-07-08 14:49:14.763 -- auc: 0.46037507519885923, Δauc: 0.4488316151202749, pcoc: 1.846998421928367, Δpcoc: 1.3134494781494142, loss: 0.0016054909229278565, Δloss: 0.0015

...

2024-07-08 14:52:13.317 -- auc: 0.6607319946434189, Δauc: 0.7398138543107422, pcoc: 1.0411119348884275, Δpcoc: 0.887226402759552, loss: 0.0013786175360119407, Δloss: 0.0014072277545928956, #instance: 366000
2024-07-08 14:52:13.821 -- auc: 0.6608268612058841, Δauc: 0.7002734472104833, pcoc: 1.0412242631507043, Δpcoc: 1.0869457392856992, loss: 0.0013782743951280371, Δloss: 0.0012526848316192626, #instance: 367000
2024-07-08 14:52:14.371 -- auc: 0.660901056898854, Δauc: 0.6891374546422984, pcoc: 1.041143975717368, Δpcoc: 1.010497485437701, loss: 0.0013780696941134722, Δloss: 0.0013029444217681884, #instance: 368000
2024-07-08 14:52:14.873 -- auc: 0.6611734279878292, Δauc: 0.752679806362379, pcoc: 1.0407794353466073, Δpcoc: 0.920643130938212, loss: 0.0013781358573817949, Δloss: 0.0014024839401245118, #instance: 369000
2024-07-08 14:52:15.387 -- auc: 0.6611262592838683, Δauc: 0.646350804245541, pcoc: 1.040196216164831, Δpcoc: 0.8575565250296342, loss: 0.001378694426207929, Δloss: 0.00158480

2024-07-08 14:53:13.141 -- auc: 0.6717951494580368, Δauc: 0.6072347148736037, pcoc: 1.0344928196017276, Δpcoc: 1.2139917356627328, loss: 0.0013678020088643324, Δloss: 0.0012726784944534302, #instance: 488000
2024-07-08 14:53:13.586 -- auc: 0.6717630477668455, Δauc: 0.6475675248775865, pcoc: 1.0347778925514057, Δpcoc: 1.2073128131719737, loss: 0.0013674284269717086, Δloss: 0.001185120463371277, #instance: 489000
2024-07-08 14:53:14.034 -- auc: 0.6719016202961932, Δauc: 0.74233496454609, pcoc: 1.0347608645665203, Δpcoc: 1.0261029581869803, loss: 0.0013672278344631194, Δloss: 0.0012691380977630615, #instance: 490000
2024-07-08 14:53:14.531 -- auc: 0.672022910224457, Δauc: 0.7292047253684082, pcoc: 1.0345181150223848, Δpcoc: 0.9217609517714557, loss: 0.0013672305538552356, Δloss: 0.0013685630559921265, #instance: 491000
2024-07-08 14:53:14.970 -- auc: 0.6720874386545623, Δauc: 0.7045297372060857, pcoc: 1.034205586067599, Δpcoc: 0.8968057036399841, loss: 0.0013674150018430338, Δloss: 0.0014

24/07/08 14:53:18 INFO ArrowPythonRunner: Times: total = 246174, boot = -39166, init = 39172, finish = 246168
24/07/08 14:53:18 INFO DataWritingSparkTask: Writer for partition 0 is committing.
24/07/08 14:53:18 INFO DataWritingSparkTask: Committed partition 0 (task 38, attempt 0, stage 18.0)
24/07/08 14:53:18 INFO Executor: Finished task 0.0 in stage 18.0 (TID 38). 3256 bytes result sent to driver
24/07/08 14:53:18 INFO TaskSetManager: Finished task 0.0 in stage 18.0 (TID 38) in 246182 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (1/1)
24/07/08 14:53:18 INFO TaskSchedulerImpl: Removed TaskSet 18.0, whose tasks have all completed, from pool 
24/07/08 14:53:18 INFO DAGScheduler: ResultStage 18 (save at NativeMethodAccessorImpl.java:0) finished in 246.198 s
24/07/08 14:53:18 INFO DAGScheduler: Job 16 is finished. Cancelling potential speculative or zombie tasks for this job
24/07/08 14:53:18 INFO TaskSchedulerImpl: Killing all running tasks in stage 18: Stage finish

[2024-07-08 14:53:28.485] [info] C[0]:9 has stopped.
[2024-07-08 14:53:28.485] [info] PS job with coordinator address 172.16.14.249:51947 stopped.


[WARN] 2024-07-08 14:53:28.417 STSAssumeRoleWithWebIdentityCredentialsProvider [139953867659072] Token file must be specified to use STS AssumeRole web identity creds provider.
[2024-07-08 14:53:28.420] [info] [s3_sdk_filesys.cpp:357] Try to open S3 stream: s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/output/dev/model_out/_dense.0.running_var__dense_data.dat, read_only false
24/07/08 14:53:28 INFO PythonRunner: Times: total = 9812, boot = -2, init = 46, finish = 9768
24/07/08 14:53:28 INFO Executor: Finished task 0.0 in stage 20.0 (TID 40). 1268 bytes result sent to driver
24/07/08 14:53:28 INFO TaskSetManager: Finished task 0.0 in stage 20.0 (TID 40) in 9817 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (1/1)
24/07/08 14:53:28 INFO TaskSchedulerImpl: Removed TaskSet 20.0, whose tasks have all completed, from pool 
24/07/08 14:53:28 INFO DAGScheduler: ResultStage 20 (collect at PythonRDD.scala:180) finished in 9.820 s
24/07/08 14:53:28 INFO DAGSc

## Evaluate the Model

To evaluate our model, we use the ``ms.input.read_s3_csv()`` function again to load the test dataset, passing in the column delimiter ``'\t'``.

In [35]:
test_dataset_path = ROOT_DIR + '/data/test/day_0_0.001_test.csv'
test_dataset = ms.input.read_s3_csv(spark_session, test_dataset_path, delimiter='\t', column_names=column_names)

ignore shuffle


24/07/08 14:54:50 INFO InMemoryFileIndex: It took 10 ms to list leaf files for 1 paths.


Next, we call the ``model.transform()`` method to transform the test dataset, which will add a column named ``rawPrediction`` to the test dataset representing the predicted labels. For ease of integration with Spark MLlib, ``model.transform()`` will also add a column named ``label`` to the test dataset representing the actual labels.

Like the training process, this will take several minutes and you can see the progress by looking at the output of the cell. The transformed test dataset is stored in the ``result`` variable.

In [36]:
result = model.transform(test_dataset)

[2024-07-08 14:54:50.798] [info] PS job with coordinator address 172.16.14.249:60157 started.
[2024-07-08 14:54:50.798] [info] PSRunner::RunPS: pid: 14693, tid: 24412, thread: 0x7f5b6bbca700
[2024-07-08 14:54:50.798] [info] PSRunner::RunPSCoordinator: pid: 14693, tid: 24412, thread: 0x7f5b6bbca700
[2024-07-08 14:54:50.799] [info] ActorProcess::Receiving: Coordinator pid: 14693, tid: 24415, thread: 0x7f5b889d5700
[2024-07-08 14:54:50.822] [info] C[0]:9: The coordinator has connected to 1 servers and 1 workers.
PS Coordinator node C[0]:9 is ready.


24/07/08 14:54:50 INFO SparkContext: Starting job: collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:291
24/07/08 14:54:50 INFO DAGScheduler: Got job 19 (collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:291) with 1 output partitions
24/07/08 14:54:50 INFO DAGScheduler: Final stage: ResultStage 21 (collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:291)
24/07/08 14:54:50 INFO DAGScheduler: Parents of final stage: List()
24/07/08 14:54:50 INFO DAGScheduler: Missing parents: List()
24/07/08 14:54:50 INFO DAGScheduler: Submitting ResultStage 21 (PythonRDD[53] at collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:291), which has no missing parents
24/07/08 14:54:50 INFO SparkContext: Starting job: collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:308
24/07

2024-07-08 14:55:04.372 -- auc: 0.7260201001456017, Δauc: 0.7260201001456017, pcoc: 1.2974920519467057, Δpcoc: 1.2974920519467057, loss: 0.0012255399227142335, Δloss: 0.0012255399227142335, #instance: 1000
2024-07-08 14:55:04.520 -- auc: 0.7180180442092818, Δauc: 0.7062628073770492, pcoc: 1.4016839108377133, Δpcoc: 1.5275824069976807, loss: 0.001155636191368103, Δloss: 0.0010857324600219726, #instance: 2000


2024-07-08 14:55:04.663 -- auc: 0.6995639819713894, Δauc: 0.667731948466993, pcoc: 1.3132465254692804, Δpcoc: 1.1620471246780888, loss: 0.0012122189203898112, Δloss: 0.0013253843784332274, #instance: 3000
2024-07-08 14:55:04.805 -- auc: 0.6693490445738648, Δauc: 0.5657445355191257, pcoc: 1.3714330108077437, Δpcoc: 1.5750857094923656, loss: 0.0012035389244556428, Δloss: 0.0011774989366531371, #instance: 4000
2024-07-08 14:55:04.945 -- auc: 0.6836984678565787, Δauc: 0.7542991262316416, pcoc: 1.4172476309996385, Δpcoc: 1.6421557664871216, loss: 0.001158206534385681, Δloss: 0.000976876974105835, #instance: 5000
2024-07-08 14:55:05.088 -- auc: 0.6912510673303766, Δauc: 0.7264107390177208, pcoc: 1.3887131019208416, Δpcoc: 1.2607996957055454, loss: 0.0011677659551302593, Δloss: 0.0012155630588531494, #instance: 6000
2024-07-08 14:55:05.229 -- auc: 0.688376831071978, Δauc: 0.6724341434626321, pcoc: 1.463457813423671, Δpcoc: 2.088953030736823, loss: 0.001139156460762024, Δloss: 0.00096749949455

24/07/08 14:55:07 INFO BlockManagerInfo: Removed broadcast_39_piece0 on ip-172-16-14-249.us-west-2.compute.internal:45547 in memory (size: 3.7 KiB, free: 2.8 GiB)


...

24/07/08 14:55:19 INFO ArrowPythonRunner: Times: total = 15526, boot = -37, init = 216, finish = 15347
24/07/08 14:55:19 INFO MemoryStore: Block rdd_83_0 stored as values in memory (estimated size 16.5 MiB, free 2.8 GiB)
24/07/08 14:55:19 INFO BlockManagerInfo: Added rdd_83_0 in memory on ip-172-16-14-249.us-west-2.compute.internal:45547 (size: 16.5 MiB, free: 2.8 GiB)
24/07/08 14:55:19 INFO DataWritingSparkTask: Writer for partition 0 is committing.
24/07/08 14:55:19 INFO DataWritingSparkTask: Committed partition 0 (task 55, attempt 0, stage 35.0)
24/07/08 14:55:19 INFO Executor: Finished task 0.0 in stage 35.0 (TID 55). 2202 bytes result sent to driver
24/07/08 14:55:19 INFO TaskSetManager: Starting task 1.0 in stage 35.0 (TID 56) (ip-172-16-14-249.us-west-2.compute.internal, executor driver, partition 1, PROCESS_LOCAL, 4919 bytes) taskResourceAssignments Map()
24/07/08 14:55:19 INFO TaskSetManager: Finished task 0.0 in stage 35.0 (TID 55) in 15688 ms on ip-172-16-14-249.us-west-2.co

2024-07-08 14:55:19.853 -- auc: 0.7128120425734789, Δauc: 0.7979148117299021, pcoc: 1.2530770534035787, Δpcoc: 1.0697464749619767, loss: 0.0013400177364450636, Δloss: 0.001414157983471767, #instance: 106978
2024-07-08 14:55:19.994 -- auc: 0.7131464377589933, Δauc: 0.7511324273914202, pcoc: 1.254236611247849, Δpcoc: 1.4028318016617387, loss: 0.0013382594678685855, Δloss: 0.0011501634120941163, #instance: 107978
2024-07-08 14:55:20.134 -- auc: 0.7126588261473952, Δauc: 0.6614602362684204, pcoc: 1.2531994140747427, Δpcoc: 1.1468256922329174, loss: 0.0013391117981231954, Δloss: 0.0014311447143554687, #instance: 108978
2024-07-08 14:55:20.276 -- auc: 0.71349707141302, Δauc: 0.7926447773341133, pcoc: 1.2492816076728483, Δpcoc: 0.9128280383784596, loss: 0.001340632717643528, Δloss: 0.00150637948513031, #instance: 109978
2024-07-08 14:55:20.414 -- auc: 0.7134185265663687, Δauc: 0.7088971132494448, pcoc: 1.2478539088649288, Δpcoc: 1.1025549616132464, loss: 0.0013413644540127081, Δloss: 0.001421

24/07/08 14:55:32 INFO ArrowPythonRunner: Times: total = 12830, boot = -125, init = 242, finish = 12713
24/07/08 14:55:32 INFO MemoryStore: Block rdd_83_1 stored as values in memory (estimated size 13.8 MiB, free 2.8 GiB)
24/07/08 14:55:32 INFO BlockManagerInfo: Added rdd_83_1 in memory on ip-172-16-14-249.us-west-2.compute.internal:45547 (size: 13.8 MiB, free: 2.8 GiB)
24/07/08 14:55:32 INFO DataWritingSparkTask: Writer for partition 1 is committing.
24/07/08 14:55:32 INFO DataWritingSparkTask: Committed partition 1 (task 56, attempt 0, stage 35.0)
24/07/08 14:55:32 INFO Executor: Finished task 1.0 in stage 35.0 (TID 56). 2202 bytes result sent to driver
24/07/08 14:55:32 INFO TaskSetManager: Finished task 1.0 in stage 35.0 (TID 56) in 12975 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (2/2)
24/07/08 14:55:32 INFO TaskSchedulerImpl: Removed TaskSet 35.0, whose tasks have all completed, from pool 
24/07/08 14:55:32 INFO DAGScheduler: ResultStage 35 (collect at /h

2024-07-08 14:55:32.756 -- auc: 0.7070430061189771, Δauc: 1.0, pcoc: 1.2245933448857014, Δpcoc: nan, loss: 0.0013348860736014274, Δloss: nan, #instance: 195974
[2024-07-08 14:55:32.767] [info] C[0]:9 has stopped.
[2024-07-08 14:55:32.767] [info] PS job with coordinator address 172.16.14.249:60157 stopped.


24/07/08 14:55:32 INFO PythonRunner: Times: total = 56, boot = -121, init = 163, finish = 14
24/07/08 14:55:32 INFO Executor: Finished task 0.0 in stage 36.0 (TID 57). 1268 bytes result sent to driver
24/07/08 14:55:32 INFO TaskSetManager: Finished task 0.0 in stage 36.0 (TID 57) in 61 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (1/1)
24/07/08 14:55:32 INFO TaskSchedulerImpl: Removed TaskSet 36.0, whose tasks have all completed, from pool 
24/07/08 14:55:32 INFO DAGScheduler: ResultStage 36 (collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspore/agent.py:308) finished in 0.064 s
24/07/08 14:55:32 INFO DAGScheduler: Job 34 is finished. Cancelling potential speculative or zombie tasks for this job
24/07/08 14:55:32 INFO TaskSchedulerImpl: Killing all running tasks in stage 36: Stage finished
24/07/08 14:55:32 INFO DAGScheduler: Job 34 finished: collect at /home/ec2-user/anaconda3/envs/metaspore/lib/python3.8/site-packages/metaspor

``result`` is a normal PySpark DataFrame and can be inspected by its methods.

In [37]:
result.show(5)

+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-----+--------------------+
|integer_feature_1|integer_feature_2|integer_feature_3|integer_feature_4|integer_feature_5|integer_feature_6|integer_feature_7|integer_feature_8

[2024-07-08 14:55:32.802] [info] PS job with coordinator address 172.16.14.249:60157 stopped.
ps agent deregistered for process 19400 thread 0x7f498c911740
24/07/08 14:55:32 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/07/08 14:55:32 INFO DAGScheduler: Got job 35 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/07/08 14:55:32 INFO DAGScheduler: Final stage: ResultStage 37 (showString at NativeMethodAccessorImpl.java:0)
24/07/08 14:55:32 INFO DAGScheduler: Parents of final stage: List()
24/07/08 14:55:32 INFO DAGScheduler: Missing parents: List()
24/07/08 14:55:32 INFO DAGScheduler: Submitting ResultStage 37 (MapPartitionsRDD[95] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/07/08 14:55:32 INFO MemoryStore: Block broadcast_43 stored as values in memory (estimated size 49.9 KiB, free 2.8 GiB)
24/07/08 14:55:32 INFO MemoryStore: Block broadcast_43_piece0 stored as bytes in memory (estimate

Finally, we use ``pyspark.ml.evaluation.BinaryClassificationEvaluator`` to compute test AUC.

In [38]:
import pyspark
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(result)
print('test_auc: %g' % test_auc)

24/07/08 14:55:32 INFO SparkContext: Starting job: sortByKey at BinaryClassificationMetrics.scala:189
24/07/08 14:55:32 INFO DAGScheduler: Registering RDD 104 (map at BinaryClassificationMetrics.scala:48) as input to shuffle 2
24/07/08 14:55:32 INFO DAGScheduler: Got job 36 (sortByKey at BinaryClassificationMetrics.scala:189) with 2 output partitions
24/07/08 14:55:32 INFO DAGScheduler: Final stage: ResultStage 39 (sortByKey at BinaryClassificationMetrics.scala:189)
24/07/08 14:55:32 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 38)
24/07/08 14:55:32 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 38)
24/07/08 14:55:32 INFO DAGScheduler: Submitting ShuffleMapStage 38 (MapPartitionsRDD[104] at map at BinaryClassificationMetrics.scala:48), which has no missing parents
24/07/08 14:55:32 INFO MemoryStore: Block broadcast_44 stored as values in memory (estimated size 43.7 KiB, free 2.8 GiB)
24/07/08 14:55:32 INFO MemoryStore: Block broadcast_44_piece0 stored as by

test_auc: 0.70704


24/07/08 14:55:33 INFO Executor: Finished task 0.0 in stage 45.0 (TID 67). 1448 bytes result sent to driver
24/07/08 14:55:33 INFO TaskSetManager: Finished task 0.0 in stage 45.0 (TID 67) in 141 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (1/2)
24/07/08 14:55:33 INFO Executor: Finished task 1.0 in stage 45.0 (TID 68). 1448 bytes result sent to driver
24/07/08 14:55:33 INFO TaskSetManager: Finished task 1.0 in stage 45.0 (TID 68) in 153 ms on ip-172-16-14-249.us-west-2.compute.internal (executor driver) (2/2)
24/07/08 14:55:33 INFO TaskSchedulerImpl: Removed TaskSet 45.0, whose tasks have all completed, from pool 
24/07/08 14:55:33 INFO DAGScheduler: ResultStage 45 (collect at BinaryClassificationMetrics.scala:237) finished in 0.155 s
24/07/08 14:55:33 INFO DAGScheduler: Job 38 is finished. Cancelling potential speculative or zombie tasks for this job
24/07/08 14:55:33 INFO TaskSchedulerImpl: Killing all running tasks in stage 45: Stage finished
24/07/08 14:55:33

When all computations are done, we should call the ``stop()`` method of ``spark_session`` to make sure all the resources are released.

In [39]:
spark_session.stop()

24/07/08 14:55:33 INFO SparkUI: Stopped Spark web UI at http://ip-172-16-14-249.us-west-2.compute.internal:4040
24/07/08 14:55:33 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/07/08 14:55:33 INFO MemoryStore: MemoryStore cleared
24/07/08 14:55:33 INFO BlockManager: BlockManager stopped
24/07/08 14:55:33 INFO BlockManagerMaster: BlockManagerMaster stopped
24/07/08 14:55:33 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/07/08 14:55:33 INFO SparkContext: Successfully stopped SparkContext


## Summary

We illustrated how to train and evaluate neural network model in MetaSpore. Users familiar with PyTorch and Spark MLlib should get started easily, which is the design goal of MetaSpore.